In [ ]:
# Install required packages
%pip install boto3 python-dotenv tavily-python


In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import boto3
from tavily import TavilyClient

# Load environment variables
load_dotenv()

# Configuration
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

# Initialize clients
bedrock_client = None
if AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY:
    try:
        bedrock_client = boto3.client(
            'bedrock-runtime',
            aws_access_key_id=AWS_ACCESS_KEY_ID,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
            region_name=AWS_REGION
        )
    except Exception as e:
        print(f"AWS Bedrock setup error: {e}")

tavily_client = TavilyClient(api_key=TAVILY_API_KEY) if TAVILY_API_KEY else None

print("🤖 Tavily Search Agent - AWS Bedrock Setup")
print("=" * 50)
print(f"☁️  AWS Region: {AWS_REGION}")
print(f"🔑 AWS Credentials: {'✅ Configured' if AWS_ACCESS_KEY_ID else '❌ Missing'}")
print(f"🧠 Bedrock Client: {'✅ Ready' if bedrock_client else '❌ Failed'}")
print(f"🔍 Tavily API: {'✅ Configured' if TAVILY_API_KEY else '❌ Missing'}")

if not AWS_ACCESS_KEY_ID:
    print("\n⚠️  Add AWS_ACCESS_KEY_ID to your .env file")
if not AWS_SECRET_ACCESS_KEY:
    print("\n⚠️  Add AWS_SECRET_ACCESS_KEY to your .env file")
if not TAVILY_API_KEY:
    print("\n⚠️  Add TAVILY_API_KEY to your .env file")

if bedrock_client and tavily_client:
    print("\n🚀 Ready to create Claude-powered search agent!")


In [ ]:
class ClaudeTavilyAgent:
    """Claude-powered agent with Tavily search capabilities via AWS Bedrock."""
    
    def __init__(self, bedrock_client, tavily_client):
        self.bedrock_client = bedrock_client
        self.tavily_client = tavily_client
        self.conversation_history = []
        
    def get_tool_definitions(self):
        """Define Tavily search tools for Claude."""
        return [
            {
                "name": "tavily_search",
                "description": "Search the web for comprehensive information on any topic. Returns multiple search results with URLs, titles, and content snippets.",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string", 
                            "description": "The search query to execute"
                        },
                        "max_results": {
                            "type": "integer",
                            "description": "Maximum number of results to return (default: 5, max: 10)",
                            "default": 5
                        }
                    },
                    "required": ["query"]
                }
            },
            {
                "name": "tavily_qna_search", 
                "description": "Get a direct, concise answer to a specific question. Perfect for factual questions that need quick answers.",
                "input_schema": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The question to get an answer for"
                        }
                    },
                    "required": ["query"]
                }
            }
        ]
    
    def execute_search(self, tool_name, tool_input):
        """Execute Tavily search functions."""
        try:
            if tool_name == "tavily_search":
                query = tool_input["query"] 
                max_results = tool_input.get("max_results", 5)
                
                response = self.tavily_client.search(
                    query=query,
                    max_results=min(max_results, 10)  # Cap at 10 for performance
                )
                
                return {
                    "status": "success",
                    "query": query,
                    "results_count": len(response.get("results", [])),
                    "results": response.get("results", [])
                }
            
            elif tool_name == "tavily_qna_search":
                query = tool_input["query"]
                answer = self.tavily_client.qna_search(query=query)
                
                return {
                    "status": "success",
                    "query": query, 
                    "answer": answer
                }
            
            return {"status": "error", "message": f"Unknown tool: {tool_name}"}
        
        except Exception as e:
            return {"status": "error", "message": str(e)}
    
    def chat(self, user_message, model_id="anthropic.claude-3-5-sonnet-20241022-v2:0"):
        """Chat with Claude - it will search the web when needed."""
        
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": user_message})
        
        # Build conversation for Claude
        system_prompt = f"""You are a helpful AI assistant with real-time web search capabilities via Tavily.
Current date: {datetime.now().strftime('%Y-%m-%d')}

You have access to these search tools:
- tavily_search: For comprehensive research with multiple sources and URLs
- tavily_qna_search: For direct factual answers to specific questions

When users ask questions that require current information, recent events, or factual data that might have changed, use the appropriate search tool. Always provide accurate, helpful responses based on the search results.

Be intelligent about when to search - not every question needs a web search."""
        
        # Prepare messages for Claude (exclude system from messages array)
        messages = []
        for msg in self.conversation_history:
            if msg["role"] != "system":
                messages.append(msg)
        
        try:
            # Call Claude via Bedrock with tool support
            request_body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4000,
                "system": system_prompt,
                "messages": messages,
                "tools": self.get_tool_definitions()
            }
            
            response = self.bedrock_client.invoke_model(
                modelId=model_id,
                body=json.dumps(request_body)
            )
            
            response_body = json.loads(response['body'].read())
            
            # Handle Claude's response
            if response_body.get('content'):
                content = response_body['content'][0]
                
                # Check if Claude wants to use a tool
                if content.get('type') == 'tool_use':
                    tool_name = content['name']
                    tool_input = content['input']
                    tool_use_id = content['id']
                    
                    print(f"🔍 Claude is searching: {tool_input.get('query', 'N/A')}")
                    
                    # Execute the search
                    search_result = self.execute_search(tool_name, tool_input)
                    
                    # Add Claude's tool use to conversation
                    self.conversation_history.append({
                        "role": "assistant",
                        "content": [
                            {
                                "type": "tool_use",
                                "id": tool_use_id,
                                "name": tool_name,
                                "input": tool_input
                            }
                        ]
                    })
                    
                    # Add tool result to conversation
                    self.conversation_history.append({
                        "role": "user",
                        "content": [
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_use_id,
                                "content": json.dumps(search_result)
                            }
                        ]
                    })
                    
                    # Get Claude's final response with search results
                    final_request = {
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 4000,
                        "system": system_prompt,
                        "messages": [msg for msg in self.conversation_history if msg["role"] != "system"],
                        "tools": self.get_tool_definitions()
                    }
                    
                    final_response = self.bedrock_client.invoke_model(
                        modelId=model_id,
                        body=json.dumps(final_request)
                    )
                    
                    final_body = json.loads(final_response['body'].read())
                    
                    if final_body.get('content') and final_body['content'][0].get('text'):
                        final_text = final_body['content'][0]['text']
                        self.conversation_history.append({"role": "assistant", "content": final_text})
                        return final_text
                    else:
                        return "I found some information but had trouble processing it."
                
                elif content.get('type') == 'text':
                    # Direct text response - no tool use needed
                    response_text = content['text']
                    self.conversation_history.append({"role": "assistant", "content": response_text})
                    return response_text
            
            return "I'm having trouble processing your request."
        
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self.conversation_history.append({"role": "assistant", "content": error_msg})
            return error_msg
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        print("🧹 History cleared!")

# Initialize Claude agent
if bedrock_client and tavily_client:
    agent = ClaudeTavilyAgent(bedrock_client, tavily_client)
    print("✅ Claude + Tavily Search Agent ready!")
else:
    agent = None
    print("❌ Agent setup failed - check AWS credentials and Tavily API key")


In [ ]:
# Example 1: Current Events with Claude
if agent:
    print("🔍 Example 1: Current AI News")
    print("=" * 40)
    response = agent.chat("What are the latest developments in AI in 2024?")
    print(f"\n🧠 Claude: {response}")
else:
    print("❌ Agent not available - check AWS credentials and Tavily API key")


In [ ]:
# Example 2: Factual Question with Claude
if agent:
    print("\n" + "="*50)
    print("❓ Example 2: Quick Fact Check")
    print("=" * 40)
    response = agent.chat("What is the current price of Bitcoin?")
    print(f"\n🧠 Claude: {response}")
else:
    print("❌ Agent not available")


In [ ]:
# Interactive Chat with Claude - Try your own questions!
def start_chat():
    """Start interactive chat with Claude agent."""
    if not agent:
        print("❌ Agent not available - check AWS credentials and Tavily API key")
        return
    
    print("\n🧠 Interactive Claude + Tavily Search Agent")
    print("=" * 55)
    print("Ask me anything! Claude will search the web for current information.")
    print("Powered by: AWS Bedrock (Claude 3.5 Sonnet) + Tavily Search")
    print("Commands: 'clear' to reset, 'quit' to exit")
    print("-" * 55)
    
    while True:
        try:
            question = input("\n👤 You: ").strip()
            
            if question.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
            elif question.lower() == 'clear':
                agent.clear_history()
                continue
            elif not question:
                continue
            
            print("\n🧠 Claude is thinking...")
            response = agent.chat(question)
            print(f"\n🧠 Claude: {response}")
            
        except KeyboardInterrupt:
            print("\n👋 Chat ended!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")

# Uncomment the line below to start interactive chat
# start_chat()

print("💡 Uncomment the last line above to start interactive chat with Claude!")


## 🔧 Setup Instructions

### Quick Start:
1. **AWS Bedrock Access**:
   - Ensure you have AWS credentials configured
   - Claude 3.5 Sonnet access enabled in your AWS region
   - AWS credentials already configured in your environment

2. **Get Tavily API Key**:
   - Sign up at [tavily.com](https://tavily.com) (free tier available)

3. **Configure Environment**:
   ```bash
   # In your .env file (AWS creds already set):
   AWS_ACCESS_KEY_ID=your_aws_access_key
   AWS_SECRET_ACCESS_KEY=your_aws_secret_key
   AWS_REGION=us-east-1
   TAVILY_API_KEY=tvly-YOUR_KEY_HERE
   ```

4. **Run the notebook** - Claude will automatically search when needed!

### How it Works:
- **Claude 3.5 Reasoning**: Advanced reasoning capabilities via AWS Bedrock
- **Intelligent Tool Selection**: Claude decides when and how to search
- **Real-time Results**: Live data from Tavily's search API
- **AWS Infrastructure**: Scalable, reliable cloud-based AI

### Example Questions to Try:
- "What's happening with Tesla stock today?"
- "Who won the latest Nobel Prize in Physics?"
- "What are the current trends in renewable energy?"
- "Tell me about recent developments in quantum computing"
- "What are the top tech news stories this week?"

### Why Claude + Bedrock?
- 🧠 **Superior Reasoning**: Claude's advanced reasoning capabilities
- ☁️ **AWS Infrastructure**: Enterprise-grade reliability and scale
- 🔒 **Data Privacy**: Your data stays in your AWS environment
- 💰 **Cost Effective**: Pay only for what you use
- 🚀 **High Performance**: Fast response times with Bedrock
